# PyTorch 基礎教學
## 教學目標

這份教學的目標是介紹基礎 PyTorch 語法，幫助同學學習未來用來撰寫深度學習模型的函式庫。

## 簡介

根據 [PyTorch 官方網站](https://pytorch.org/)（穩定版 v2.6.0）：

> PyTorch is an open source machine learning framework that accelerates the path from research prototyping to production deployment.
>
> PyTorch 是一個開源的機器學習框架，能夠幫助加速從研究原型到商業應用的轉換過程。

![PyTorch usage statistics](https://thegradient.pub/content/images/2019/10/ratio_medium-1.png)

根據[統計](https://thegradient.pub/state-of-ml-frameworks-2019-pytorch-dominates-research-tensorflow-dominates-industry/)，PyTorch 在各大機器學習會議使用率逐年上升，使用者選擇 PyTorch 的原因為：

- 簡潔，使用 `Python` 作為介面，且操作方法與 `NumPy` 相似
- 好用的函式介面，沒有過多的抽象化
- 執行效能佳

In [ ]:
import numpy as np
import torch

In [ ]:
print(
    # 確認 torch 的版本
    f'PyTorch version {torch.__version__}\n' +
    # 確認是否有 GPU 裝置
    f'GPU-enabled installation? {torch.cuda.is_available()}'
)

## 張量宣告

在 `torch` 中陣列稱為張量（Tensor），創造張量的語法為 `torch.tensor([value1, value2, ...])`。

- 每個 `torch.Tensor` 都有不同的**數值型態屬性** `torch.Tensor.dtype`
    - 必須透過 `torch.Tensor.dtype` 取得，無法透過 `type()` 取得
- 可以指定型態
    - 透過參數 `dtype` 指定型態
    - 透過 `torch.LongTensor` 創造整數，預設為 `torch.int64`
    - 透過 `torch.FloatTensor` 創造浮點數，預設為 `torch.float32`
- [不同 Tensor 型態比較](https://pytorch.org/docs/stable/tensors.html)

|`torch` 型態|`numpy` 型態|C 型態|範圍|
|-|-|-|-|
|`torch.int8`|`numpy.int8`|`int_8`|-128~127|
|`torch.int16`|`numpy.int16`|`int_16`|-32768~32767|
|`torch.int32`|`numpy.int32`|`int_32`|-2147483648~2147483647|
|`torch.int64`|`numpy.int64`|`int_64`|-9223372036854775808~9223372036854775807|
|`torch.float32`|`numpy.float32`|`float`||
|`torch.float64`|`numpy.float64`|`double`||

- 每個 `torch.Tensor` 都有**維度屬性** `torch.Size`
    - 呼叫 `torch.Tensor.size()` 來取得維度屬性
    - `torch.Tensor.size` 本質是 `tuple`
    - 張量維度愈高，`len(torch.Tensor.size)` 數字愈大
- 可以使用 `torch.Tensor.reshape` 或 `torch.Tensor.view` 進行維度變更
    - 變更後的維度必須要與變更前的維度乘積相同
    - 變更後的內容為 **shallow copy**

In [ ]:
# 張量宣告

# 宣告 Tensor 變數
t1 = torch.tensor([1, 2, 3])
# 輸出 Tensor
print(t1)
# 輸出 True
print(type(t1) == torch.Tensor)
# 輸出 torch.int64
print(t1.dtype)
print()

# 宣告 Tensor 變數
t2 = torch.tensor([1., 2., 3.])
# 輸出 Tensor
print(t2)
# 輸出 True
print(type(t2) == torch.Tensor)
# 輸出 torch.float32
print(t2.dtype)
print()

# 各種 dtype
# 輸出 torch.int8
print(torch.tensor([1, 2], dtype=torch.int8).dtype)
x = torch.tensor([1, 2], dtype=torch.int8)
print(x)
try:
    x[0] = 128
except Exception as e:
    # int8 的範圍為 -128 ~ 127
    # RuntimeError: value cannot be converted to type int8_t without overflow
    print(e)
print()

In [ ]:
# 宣告 LongTensor 變數 -> 通常 label 會使用這種型態，因為 label 通常是整數，在 torch 訓練的錯誤時可能會看到類似這種錯誤：
# RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'
# 此時就要記得檢查是否有使用到 LongTensor
t3 = torch.LongTensor([1, 2, 3])
# 輸出 torch.int64
print(t3.dtype)

# 宣告 FloatTensor 變數
t4 = torch.FloatTensor([1, 2, 3])
# 輸出 torch.float32
print(t4.dtype)

### 隨機（Random）

創造出新的張量，所有數值皆為**隨機決定**，必須**事先指定張量維度**。

|函數|意義|用途|備註|
|-|-|-|-|
|`torch.empty`|創造隨機未初始化張量|已確認維度，尚未確認數值|無法控制隨機|
|`torch.rand`|創造隨機浮點數張量，並符合均勻分佈|需要隨機浮點數時|透過均勻分佈決定亂數，範圍介於 0 到 1之間|
|`torch.randn`|創造隨機浮點數張量，並符合常態分佈|需要符合常態分佈的隨機浮點數時|透過常態分佈決定亂數，$\mu = 0$ 且 $\sigma = 1$|
|`torch.randint`|創造隨機整數張量|需要隨機整數時|透過均勻分佈決定亂數，可以控制隨機範圍|

In [ ]:
# 隨機

# 隨機創造維度為 (2, 3) 的張量，數值為無法控制範圍的浮點
print(torch.empty((2, 3)))
print()

# 隨機創造維度為 (2, 3) 的張量，數值為介於 0 到 1 之間的浮點
print(torch.rand(2, 3))
print()

# 隨機創造維度為 (2, 3) 的張量，數值為介於 0 到 10 之間的浮點
print(torch.rand(2, 3) * 10)
print()

# 隨機創造維度為 (2, 3) 的張量，數值為介於 -5 到 5 之間的浮點數
print(torch.rand(2, 3) * 10 - 5)
print()

# 隨機創造維度為 (2, 3) 的張量，分佈為平均值為 0 標準差為 1 的常態分佈
print(torch.randn(2, 3))
print()

# 隨機創造維度為 (2, 3) 的張量，數值為介於 -5 到 5 之間的浮點數
print(torch.randint(-5, 5, size=(2, 3)))

### 指定數值（Filled In）

**快速創造**擁有特定數值的張量，必須**事先指定張量維度**。

|函數|意義|用途|
|-|-|-|
|`torch.zeros`|創造指定維度大小的張量，所有數值初始化為 0|快速初始化|
|`torch.zeros_like`|複製指定張量的維度，創造出新的張量，所有數值初始化為 0|複製張量並初始化|
|`torch.ones`|創造指定維度大小的張量，所有數值初始化為 1|快速初始化|
|`torch.ones_like`|複製指定張量的維度，創造出新的張量，所有數值初始化為 1|複製張量並初始化|
|`torch.full`|創造指定維度大小的張量，所有數值初始化為指定數值|快速初始化|
|`torch.full_like`|複製指定張量的維度，創造出新的張量，所有數值初始化為指定數值|複製張量並初始化|
|`torch.eye`|創造單位矩陣|矩陣微分|
|`torch.arange`|列舉數字|等同於 `list(range(value))`|

In [ ]:
# 指定數值
# 創造維度為 (2, 3) 的張量，並初始化為 0
print(torch.zeros((2, 3)))
print()

# 宣告 Tensor 變數
t5 = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
])
# 複製張量 t5 的維度，創造出新的張量，並初始化為 0
print(torch.zeros_like(t5))
print()

# 創造維度為 (3, 4) 的張量，並初始化為 1
print(torch.ones((3, 4)))
print()

# 宣告 Tensor 變數
t6 = torch.tensor([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
])
# 複製張量 t6 的維度，創造出新的張量，並初始化為 1
# like 的概念就是「模仿我丟給你的這個 tensor 的維度」
print(torch.ones_like(t6))
print()

# 創造維度為 (5, 6) 的張量，並初始化為 420
print(torch.full((5, 6), 420))
print()

# 宣告 Tensor 變數
t7 = torch.tensor([
    [1, 2, 3, 4, 5, 6],
    [7, 8, 9, 10, 11, 12],
    [13, 14, 15, 16, 17, 18],
    [19, 20, 21, 22, 23, 24],
    [25, 26, 27, 28, 29, 30]
])
# 複製張量 t7 的維度，創造出新的張量，並初始化為 69
print(torch.full_like(t7, 69))

In [ ]:
# 創造 3x3 單位矩陣
print(torch.eye(3))
print()

# 從 0 列舉至 10，但不包含 10
print(torch.arange(10))
print()

# 從 6 列舉至 9，但不包含 9
print(torch.arange(6, 9))
print()

# 從 4 遞增至 20，但不包含 20，每次遞增 7
print(torch.arange(4, 20, 7))

### 從 numpy 轉換

可以使用 `torch.tensor()` 將 `numpy.ndarray` 轉換成 `torch.Tensor`；
使用 `torch.numpy()` 將 `torch.Tensor` 轉換成 `numpy.ndarray`。

In [ ]:
# 從 numpy 轉換

# 宣告 ndarray 變數
arr1 = np.array([1., 2., 3.])
# 將 numpy.ndarray 轉換為 torch.Tensor
t8 = torch.tensor(arr1)
# 將 torch.Tensor 轉換為 numpy.ndarray
arr2 = t8.numpy()

print((
    f'original numpy.ndarray: {arr1}, dtype: {arr1.dtype}\n' +
    f'converted torch.Tensor: {t8}, dtype: {t8.dtype}\n' +
    f'converted numpy.ndarray: {arr2}, dtype: {arr2.dtype}'
))

## 張量取值

與 `numpy` 語法概念相似。

- 使用 `torch.Tensor[位置]` 來取得 `torch.Tensor` 中指定位置的值
    - 若為**多個維度**的張量，則使用 `tuple` 來取得指定位置的值
    - 若位置為**負數**，則等同於反向取得指定位置的值
    - 取出的值會以 `torch.Tensor.dtype` 的形式保留
- 使用 `torch.Tensor[起始位置:結束位置]` 來取得 `torch.Tensor` 中的部分**連續**值
    - **包含起始位置**的值
    - **不包含結束位置**的值
    - 取出的值會以 `torch.Tensor` 的形式保留
- 使用 `torch.Tensor[iterable]`（例如 `list`, `tuple` 等）來取得**多個** `torch.Tensor` 中的值
    - 取出的值會以 `torch.Tensor` 的形式保留
- 使用判斷式來取得 `torch.Tensor` 中的部份資料
    - 經由判斷式所得結果也為 `torch.Tensor`
    - 判斷式所得結果之 `torch.Tensor.dtype` 為**布林值** `bool`（`True` 或 `False`）
    - 取出的值會以 `torch.Tensor` 的形式保留

In [ ]:
# 張量取值

# 宣告 Tensor 變數
t9 = torch.tensor([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [9, 10, 11],
])

# 輸出張量 t9 中的第 0 個位置的值 [0, 1, 2]
print(t9[0])
# 輸出張量 t9 中的第 1 個位置的值 [3, 4, 5]
print(t9[1])
# 輸出張量 t9 中的第 1 個位置的值 [6, 7, 8]
print(t9[2])
# 輸出張量 t9 中的第 -2 個位置的值 [6, 7, 8]
print(t9[-2])
# 輸出張量 t9 中的第 -1 個位置的值 [9, 10, 11]
print(t9[-1])
print()

# 輸出張量 t9 中的第 [0, 0] 個位置的值 0
print(t9[0, 0])
# 輸出張量 t9 中的第 [0, 1] 個位置的值 1
print(t9[0, 1])
# 輸出張量 t9 中的第 [1, 1] 個位置的值 4
print(t9[1, 1])
# 輸出張量 t9 中的第 [1, 2] 個位置的值 5
print(t9[1, 2])
# 輸出張量 t9 中的第 [-1, -1] 個位置的值 11
print(t9[-1, -1])
# 輸出張量 t9 中的第 [-1, -2] 個位置的值 10
print(t9[-1, -2])
# 輸出張量 t9 中的第 [-2, -1] 個位置的值 8
print(t9[-2, -1])

In [ ]:
# 取連續值

# 宣告 Tensor 變數
t10 = torch.tensor([
    0, 10, 20, 30, 40,
    50, 60, 70, 80, 90
])

# 輸出張量 t10 位置 0, 1, 2 但是不含位置 3 的值 [0, 10, 20]
print(t10[0:3])
# 輸出張量 t10 位置 7, 8, 9 的值 [70, 80, 90]
print(t10[7:])
# 輸出張量 t10 位置 0, 1 但是不含位置 2 的值 [0, 10]
print(t10[:2])
# 輸出張量 t10 所有位置的值 [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
print(t10[:])
print()

# 宣告 Tensor 變數
t11 = torch.tensor([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [9, 10, 11],
])

# 輸出張量 t11 位置 0, 1, 但是不含位置 2 的值 [[0, 1, 2], [3, 4, 5]]
print(t11[0:2])
print()

# 輸出張量 t11 位置 1, 2, 3 的值 [[3, 4, 5], [6, 7, 8], [9, 10, 11]]
print(t11[1:])
print()

# 輸出張量 t11 位置 0 但是不含位置 1 的值 [[0, 1, 2]]
print(t11[:1])
print()

# 輸出張量 t11 位置 0 但是不含位置 1 的值 [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
print(t11[:])

In [ ]:
# 使用 iterable 取得多個值

# 宣告 Tensor 變數
t12 = torch.tensor([
    0, 10, 20, 30, 40,
    50, 60, 70, 80, 90
])

# 輸出張量 t12 中偶數位置的值 [0, 20, 40, 60, 80]
print(t12[[0, 2, 4, 6, 8]])
print()
# 輸出張量 t12 中奇數位置的值 [10, 30, 50, 70, 90]
print(t12[[1, 3, 5, 7, 9]])
print()

# 宣告 Tensor 變數
t13 = torch.tensor([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
])

# 輸出張量 t13[0] 與 t13[1] 的值 [[1, 2, 3, 4] [5, 6, 7, 8]]
print(t13[[0, 1]])
print()
# 輸出張量 t13[0, 2] 與 t13[1, 3] 的值 [3, 8]
print(t13[[0, 1], [2, 3]])

In [ ]:
# 將張量 t13 位置 0 的所有數值改成 1995
t13[0] = 1995
print(t13)
print()

# 將張量 t13 位置 [0, 1] 的數值改成 10
t13[0, 1] = 10
print(t13)
print()

# 將張量 t13 位置 [[0, 1]] 的數值改成 10
t13[[0, 1]] = -999
print(t13)

In [ ]:
# 判斷式取值

# 宣告 Tensor 變數
t15 = torch.tensor([
    0, 10, 20, 30, 40,
    50, 60, 70, 80, 90
])

# 輸出每個值是否大於 50 的 `torch.Tensor`
print(t15 > 50)
# 輸出 torch.bool
print((t15 > 50).dtype)
# 輸出大於 50 的值 [60, 70, 80, 90]
print(t15[t15 > 50])
# 輸出除以 20 餘數為 0 的值 [0, 20, 40, 60, 80]
print(t15[t15 % 20 == 0])

## 張量運算

### 純量運算（Scalar Operation）

對張量內所有數值與單一純量（Scalar）進行相同計算。

|符號|意義|
|-|-|
|`torch.Tensor + scalar`|張量中的每個數值加上 `scalar`|
|`torch.Tensor - scalar`|張量中的每個數值減去 `scalar`|
|`torch.Tensor * scalar`|張量中的每個數值乘上 `scalar`|
|`torch.Tensor / scalar`|張量中的每個數值除以 `scalar`|
|`torch.Tensor // scalar`|張量中的每個數值除以 `scalar` 所得之商|
|`torch.Tensor % scalar`|張量中的每個數值除以 `scalar` 所得之餘數|
|`torch.Tensor ** scalar`|張量中的每個數值取 `scalar` 次方|

### 個別數值運算（Element-wise Operation）

若兩個張量想要進行運算，則兩個張量的**維度必須相同**（即兩張量之 `torch.size()` 相同）。

|符號|意義|
|-|-|
|`A + B`|張量 `A` 中的每個數值加上張量 `B` 中相同位置的數值|
|`A - B`|張量 `A` 中的每個數值減去張量 `B` 中相同位置的數值|
|`A * B`|張量 `A` 中的每個數值乘上張量 `B` 中相同位置的數值|
|`A / B`|張量 `A` 中的每個數值除以張量 `B` 中相同位置的數值|
|`A // B`|張量 `A` 中的每個數值除以張量 `B` 中相同位置的數值所得之商|
|`A % B`|張量 `A` 中的每個數值除以張量 `B` 中相同位置的數值所得之餘數|
|`A ** B`|張量 `A` 中的每個數值取張量 `B` 中相同位置的數值之次方|

### 個別數值函數運算（Element-wise Functional Operation）

若想對張量中的**所有數值**進行**相同函數運算**，必須透過 `torch` 提供的介面進行。

|函數|意義|
|-|-|
|`torch.sin`|張量中的每個數值 $x$ 計算 $\sin(x)$|
|`torch.cos`|張量中的每個數值 $x$ 計算 $\cos(x)$|
|`torch.tan`|張量中的每個數值 $x$ 計算 $\tan(x)$|
|`torch.exp`|張量中的每個數值 $x$ 計算 $e^{x}$|
|`torch.log`|張量中的每個數值 $x$ 計算 $\log x$
|`torch.ceil`|張量中的每個數值 $x$ 計算 $\left\lceil x \right\rceil$
|`torch.floor`|張量中的每個數值 $x$ 計算 $\left\lfloor x \right\rfloor$

In [ ]:
# 純量運算(Scalar Operation)

# 宣告 Tensor 變數
t16 = torch.tensor([
    [0, 10, 20],
    [30, 40, 50],
    [60, 70, 80],
    [90, 100, 110],
])

# 輸出張量 t16
print(t16)
print()
# 對張量 t16 所有數值加 5
print(t16 + 5)
print()
# 對張量 t16 所有數值減 4
print(t16 - 4)
print()
# 對張量 t16 所有數值乘 3
print(t16 * 3)
print()
# 對張量 t16 所有數值除以 10
print(t16 / 10)
print()
# 對張量 t16 所有數值除以 10 所得整數部份
print(t16 // 10)
print()
# 對張量 t16 所有數值除以 7 得到餘數
print(t16 % 7)
print()
# 對張量 t16 所有數值取 2 次方
print(t16 ** 2)

In [ ]:
# 個別數值運算

# 宣告 Tensor 變數
t17 = torch.tensor([
    [1, 2, 3],
    [4, 5, 6]
])

# 宣告 Tensor 變數
t18 = torch.tensor([
    [6, 5, 4],
    [3, 2, 1]
])

# 張量相加
print(t17 + t18)
print()
# 張量相減
print(t17 - t18)
print()
# 張量相乘
print(t17 * t18)
print()
# 張量相除
print(t17 / t18)
print()
# 張量相除取商
print(t17 // t18)
print()
# 張量相除取餘數
print(t17 % t18)
print()
# 張量 A 取張量 B 次方
print(t17 ** t18)

### 🚧 **張量自動擴充（Broadcasting)**

若張量 `A` 的維度為 `(a1, a2, ..., an)`（即 `A.size() == (a1, a2, ..., an)`），則張量 `B` 在滿足以下其中一種條件時即可與張量 `A` 進行運算：

- 張量 `B` 與張量 `A` 維度完全相同（即 `B.size() == (a1, a2, ..., an)`）
- 張量 `B` 為純量（即 `B.size() == (1,)`）
- 張量 `B` 的維度為 `(b1, b2, ..., bn)`，若 `ai != bi`，則 `ai == 1` 或 `bi == 1`
    - 從**最後**一個維度開始比較
    - 如果有任何一個維度無法滿足前述需求，則會得到 `ValueError`

In [ ]:
# 張量自動擴充

# 宣告 Tensor 變數
t19 = torch.tensor([
    [
        [1, 2],
        [3, 4],
        [5, 6],
    ],
    [
        [7, 8],
        [9 ,10],
        [11, 12]
    ]
])

# 宣告 Tensor 變數
t20 = torch.tensor([
    [
        [1],
        [1],
        [1]
    ],
    [
        [2],
        [2],
        [2]
    ],
])

# 輸出張量 t19 維度
print(t19.size())
# 輸出張量 t20 維度
print(t20.size())
print()
# 張量 t19 與張量 t19 維度相同，所以可以直接運算
print(t19 + t19)
print()
# 張量 t19 與張量 t20 可以擴充成相同維度，所以可以運算
print(t19 + t20)

In [ ]:
# 張量 t17 與張量 t18 可以擴充成相同維度，所以可以運算
print(t19 + t20)
t20_like = torch.tensor([
    [
        [1, 1],
        [1, 1],
        [1, 1]
    ],
    [
        [2, 2],
        [2, 2],
        [2, 2]
    ],
])
print()

print(t20_like.size())
print()

# 輸出 True，因為 t20_like 與 t20 擴充後維度相同
print(torch.equal(t20 + t19, t20_like + t19))

### 賦值（Assignment）

使用 `=` 賦與指定位置數值。可以使用 `iterable` 一次指定多個位置。

|符號|意義|
|-|-|
|`=`|賦值|
|`+=`|進行加法後賦值|
|`-=`|進行減法後賦值|
|`*=`|進行乘法後賦值|

In [ ]:
# 使用 = 的賦值操作

t21 = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
print(t21)
print()

# 將張量 t21 位置 0 的所有數值加上 1995
t21[0] += 1995
print(t21)
print()

# 將張量 t21 位置 [0, 1] 的所有數值減掉 10
t21[0, 1] -= 10
print(t21)
print()

# 將張量 t21 位置 [2, 1] 與 [0, 2] 的所有數值乘上 12
t21[[2, 0], [1, 2]] *= 12
print(t21)

## 高維張量運算
矩陣等同於是維度為 2 的張量。
而高維度的張量運算等同於**固定大部分的維度**，只使用**其中的兩個維度進行計算**。

### 張量乘法（Tensor Multiplication）

例如：以 $A.\text{size}() = (4, 3)$ 與 $B.\text{size}() = (3, 2)$ 來說，$(A \times B).\text{size}() = (4, 2)$。

例如：以 $A.\text{size}() = (5, 4, 3)$ 與 $B.\text{size}() = (5, 3, 2)$ 來說，$(A \times B).\text{size}() = (5, 4, 2)$。

例如：以 $A.\text{size}() = (1995, 10, 12, 5, 4, 3)$ 與 $B.\text{size}() = (1995, 10, 12, 5, 3, 2)$ 來說，$(A \times B).\text{size}() = (1995, 10, 12, 5, 4, 2)$。

In [ ]:
# 張量乘法

# 宣告 Tensor 變數
t22 = torch.ones(5, 4, 3)
# 宣告 Tensor 變數
t23 = torch.ones(5, 3, 2)
# 進行張量乘法
t24 = torch.matmul(t22, t23)

# 輸出張量 t22 的維度
print(t22.size())
# 輸出張量 t23 的維度
print(t23.size())
# 輸出張量 t24 的維度
print(t24.size())

In [ ]:
# 矩陣乘法

# 宣告 Tensor 變數
t25 = torch.tensor([[1, 2, 3], [4, 5, 6]])

t26 = torch.full((3,2), 2)
print(t26)
print()

t27 = torch.matmul(t25, t26)
print(t27)

In [ ]:
# 張量轉置

# 宣告 Tensor 變數
t28 = torch.ones(5, 4, 3)

# 輸出轉置維度 1 與 2 後的維度
print(torch.transpose(t28, 1, 2).size())
# 輸出轉置維度 1 與 2 後的維度
print(t28.transpose(1, 2).size())

# 輸出轉置維度 0 與 2 後的維度
print(torch.transpose(t28, 0, 2).size())
# 輸出轉置維度 0 與 2 後的維度
print(t28.transpose(0, 2).size())